In [41]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html


In [42]:
# Build App
app = JupyterDash(__name__)

app.layout = html.Div([
    
    html.Div(
        className="navbar",
        children=[ 
            html.H3('Interactive Data Analytics Dashboard for RDF Knowledge Graphs', className="navbar--title"),
            html.Div(
                className="navbar--btns",
                children=[
                    html.Span(
                        className="navbar--btn_left",
                        children="Dashboard"),
                    html.Span(
                        className="navbar--btn_left",
                        children="Tables"),
                    html.Span(
                        className="navbar--btn_left",
                        children="Charts"),
                    html.Span(
                        className="navbar--btn_left",
                        children="Query Samples"),
                    html.Span(
                        className="navbar--btn_right",
                        children="Upload"),
                    html.Span(
                        className="navbar--btn_right",
                        children="Download"),]),

   
    html.Div(
        className="querybox",
        children=[
            html.H4("SPARQL Query", className="querybox--title"),
            html.Textarea(className="querybox--textarea"),
            html.Button("SUBMIT", className="querybox--btn")])
      ])
       
])#lay-out div    
    


In [43]:
# Run app and display result inline in the notebook
app.run_server(mode='inline')

# Run app and display result on localhost
#app.run_server(mode='external')